### Quick visualisation of the datas, 
download data here : https://conservancy.umn.edu/handle/11299/257209
and the csv form

In [2]:
import pandas as pd
import numpy as np

import sunpy.coordinates  # NOQA
import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
from functions_AIA import *

In [5]:
data=load_data() #Function in AIA.py to load positive data

In [6]:
data=data.drop(columns="velocity")
data

KeyError: "['velocity'] not found in axis"

In [6]:
data[data["duration"]==data["duration"].min()]

,#sjh_ID,date,SOL_event,duration,basepoint_X,std_X,basepoint_Y,std_Y,basepoint_X_longitude,basepoint_Y_latitude,max_height,upper_H,lower_H,avg_width,std_width,sigma,flag
126,sjh_2011-12-29T06_1,2011-12-29T06:50:56.000000,SOL2011-12-29T06:30:00L156C103,5.4,730.145,0.0,255.1400,0.0,50.0,13.0,160.040427,54.978985,-40.921236,93.272541,0.0,0.446007,110
173,sjh_2012-07-05T03_1,2012-07-05T03:29:07.000000,SOL2012-07-05T03:23:44L093C121,5.4,-872.636,0.0,-376.5300,0.0,-88.0,-23.0,121.109260,47.010698,-33.865288,53.496318,0.0,0.481061,110
320,sjh_2013-12-23T17_2,2013-12-23T17:39:43.000000,SOL2013-12-23T17:39:38L006C107,5.4,862.295,0.0,-284.2700,0.0,67.0,-17.0,21.164057,3.504392,-3.006559,23.676062,0.0,0.263938,110
799,sjh_2016-06-02T08_2,2016-06-02T08:35:54.000000,SOL2016-06-02T08:04:38L139C085,5.4,892.983,0.0,59.7915,0.0,70.0,3.0,38.611724,2.809068,-2.618563,11.011035,0.0,0.131036,110
814,sjh_2016-08-11T13_2,2016-08-11T13:42:54.000000,SOL2016-08-11T12:35:15L172C083,5.4,-469.017,0.0,67.5762,0.0,-30.0,9.0,61.294669,18.443509,-14.177509,30.131523,0.0,0.409102,110


In [7]:
print(data["duration"].min())
print(30*24/60)

5.4
12.0


In [8]:
date=data["date"].copy()
date=pd.to_datetime(date)
date=date.dt.date
date=pd.DataFrame({"date": date.unique()})
date

,date
0,2011-01-20
1,2011-01-21
2,2011-01-24
3,2011-01-26
4,2011-02-13
...,...
186,2016-10-28
187,2016-11-28
188,2016-12-01
189,2016-12-14


In [5]:
len(data["date"].unique())

835

In [10]:
start_date = date["date"].min()
end_date = date["date"].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='D')
full_days = pd.DataFrame({'date': date_range})
full_days=full_days[~full_days["date"].isin(date["date"])]
#outer=pd.merge(full_days, date, how="left", on="date")
#outer
full_days

,date
2,2011-01-22
3,2011-01-23
5,2011-01-25
7,2011-01-27
8,2011-01-28
...,...
2160,2016-12-19
2161,2016-12-20
2162,2016-12-21
2163,2016-12-22


In [11]:
randomized_days= full_days.sample(frac=1, random_state=42).reset_index()
randomized_days["basepoint_X"]=data["basepoint_X"]
randomized_days["basepoint_Y"]=data["basepoint_Y"]
randomized_days=randomized_days.dropna()
randomized_days=randomized_days.drop(columns="index")
randomized_days

,date,basepoint_X,basepoint_Y
0,2016-07-09,-226.577125,-956.964375
1,2016-03-23,-143.625000,386.404000
2,2013-01-15,-142.500000,389.260000
3,2012-01-30,-100.466000,387.028000
4,2016-05-29,-11.371400,475.010000
...,...,...,...
878,2013-11-20,90.741233,992.448333
879,2015-10-10,-925.214000,334.932400
880,2016-05-28,139.922000,946.276000
881,2016-07-17,-0.883805,988.594000


In [12]:
def randomize_time(date):
    np.random.seed=10
    random_hour = np.random.randint(0, 24)
    random_minute = np.random.randint(0, 60)
    #liste=[8,20,32,44,56] #Only possible seconds 12 sec intervals
    #val=np.random.randint(0, 5) 
    random_second = np.random.randint(0, 60)#liste[val]
    return date.replace(hour=random_hour, minute=random_minute, second=random_second)

In [11]:
randomized_days["date"]=randomized_days["date"].apply(randomize_time)
randomized_days['date'] = pd.to_datetime(randomized_days['date']).dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
display(randomized_days)


,date,basepoint_X,basepoint_Y
0,2016-07-09T06:55:58.000000,-226.577125,-956.964375
1,2016-03-23T11:02:29.000000,-143.625000,386.404000
2,2013-01-15T22:02:36.000000,-142.500000,389.260000
3,2012-01-30T20:03:04.000000,-100.466000,387.028000
4,2016-05-29T08:36:22.000000,-11.371400,475.010000
...,...,...,...
878,2013-11-20T01:16:35.000000,90.741233,992.448333
879,2015-10-10T13:49:47.000000,-925.214000,334.932400
880,2016-05-28T11:55:58.000000,139.922000,946.276000
881,2016-07-17T03:10:22.000000,-0.883805,988.594000


In [13]:
""" Importing HEK CJ events """

event_type = "CJ"
start_date = date["date"].min()
end_date = date["date"].max()

result = Fido.search(a.Time('2011-01-20T00:00', '2016-12-24T00:00'),
                     a.hek.EventType(event_type),)
                     #a.hek.FL.GOESCls > "M1.0",
                     #a.hek.OBS.Observatory == "GOES")

#print(result.all_colnames())
hek_results = result["hek"]
print(hek_results.colnames)

['gs_thumburl', 'comment_count', 'hpc_bbox', 'outflow_width', 'frm_humanflag', 'hgc_coord', 'event_coordsys', 'obs_levelnum', 'hpc_coord', 'event_npixels', 'gs_imageurl', 'ar_polarity', 'frm_paramset', 'hrc_coord', 'event_starttime', 'ar_mtwilsoncls', 'event_type', 'intensmin', 'obs_meanwavel', 'outflow_openingangle', 'frm_url', 'skel_chaincode', 'bound_chaincode', 'noposition', 'active', 'intensmax', 'frm_versionnumber', 'area_uncert', 'obs_dataprepurl', 'hpc_geom', 'hgc_bbox', 'intensmedian', 'chaincodetype', 'obs_channelid', 'event_clippedspatial', 'ar_noaaclass', 'SOL_standard', 'event_avg_rating', 'eventtype', 'intensunit', 'hpc_boundcc', 'event_mapurl', 'frm_contact', 'ar_penumbracls', 'intensmean', 'bound_ccstartc1', 'outflow_transspeed', 'frm_name', 'area_atdiskcenter', 'frm_identifier', 'obs_observatory', 'event_description', 'boundbox_c2ur', 'obs_firstprocessingdate', 'boundbox_c2ll', 'frm_institute', 'hrc_bbox', 'refs_orig', 'ar_mcintoshcls', 'event_maskurl', 'bound_ccstartc

In [14]:
filtered_results = hek_results['SOL_standard','event_coord1', 'event_coord2','hpc_bbox','event_starttime','event_endtime'] 
results_df = filtered_results.to_pandas()


df = results_df.drop_duplicates()

"""
 Split each event into chunks of 12 minutes (24s * 30)

 """

df['event_starttime'] = pd.to_datetime(df['event_starttime'])
df['event_endtime'] = pd.to_datetime(df['event_endtime'])

split_interval = pd.to_timedelta('12 minutes')

# Create a new DataFrame to store the split events
column_names = ['SOL_standard', 'event_coord1', 'event_coord2', 'event_starttime', 'event_endtime' ]
SOL_events = pd.DataFrame(columns=column_names)

# Iterate over the original DataFrame
for index, row in df.iterrows():
    event_start = row['event_starttime']
    event_end = row['event_endtime']
    # Generate a range of timestamps within the original event's time interval
    intervals = pd.date_range(start=event_start, end=event_end, freq=split_interval)
    
    time_stamps = intervals[0]
    for i in range((intervals.shape)[0]-1):
        SOL_events.loc[len(SOL_events.index)] = [row['SOL_standard'], 
                        row['event_coord1'], 
                        row['event_coord2'], 
                        intervals[i],
                        intervals[i+1]]


# Display the DataFrame with split events
print(SOL_events.shape)
SOL_events

/var/folders/7l/ltdxzx1s3dj_sz2kw6_5g8k00000gn/T/ipykernel_59081/1682553782.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event_starttime'] = pd.to_datetime(df['event_starttime'])
/var/folders/7l/ltdxzx1s3dj_sz2kw6_5g8k00000gn/T/ipykernel_59081/1682553782.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['event_endtime'] = pd.to_datetime(df['event_endtime'])


(4713, 5)


,SOL_standard,event_coord1,event_coord2,event_starttime,event_endtime
0,SOL2011-01-20T09:00:09L289C165,-245.000000,-1003.000000,2011-01-20 09:00:09,2011-01-20 09:12:09
1,SOL2011-01-20T09:00:09L289C165,-245.000000,-1003.000000,2011-01-20 09:12:09,2011-01-20 09:24:09
2,SOL2011-01-20T09:00:09L289C165,-245.000000,-1003.000000,2011-01-20 09:24:09,2011-01-20 09:36:09
3,SOL2011-01-20T09:00:09L289C165,-245.000000,-1003.000000,2011-01-20 09:36:09,2011-01-20 09:48:09
4,SOL2011-01-20T09:15:44L290C165,-226.577118,-956.964355,2011-01-20 09:15:44,2011-01-20 09:27:44
...,...,...,...,...,...
4708,SOL2016-12-10T01:00:06L076C076,373.000000,244.000000,2016-12-10 03:36:06,2016-12-10 03:48:06
4709,SOL2016-12-10T01:00:06L076C076,373.000000,244.000000,2016-12-10 03:48:06,2016-12-10 04:00:06
4710,SOL2016-12-10T01:00:06L076C076,373.000000,244.000000,2016-12-10 04:00:06,2016-12-10 04:12:06
4711,SOL2016-12-14T18:16:06L031C014,144.000000,952.000000,2016-12-14 18:16:06,2016-12-14 18:28:06


In [15]:
""" 
Compare each time chunk of split_df (HEK CJ events) and the original csv

"""

# Add a new column to our .csv containing the endtime of each event
csv_df = data.copy()
csv_df['date'] = pd.to_datetime(csv_df['date'])

# Calculate the end time and add a new column 'End_Time'
csv_df['end_time'] = csv_df['date'] + pd.to_timedelta(csv_df['duration'], unit='m')
csv_df

""" TO DO
- for each SOL_event, compare time intervals from split_df and csv_df
=> convert starttime and endtime of csv_df to intervals using pd.interval('starttime', 'endtime')
=> check for overlap by comparing each start_time and end_time from split_df with the jet interval (use interval.right and interval.left)
- use boolean indexing to index the intervals in split_df that do not overlap
- use those the fetch images
"""

boolean_df = pd.DataFrame(columns=['overlap'])

def intervals_overlap(start1, end1, start2, end2): 

    if (start2 <= start1) & (start1 <= end2): # overlap on the right side of event 1
        ans = True
    elif (start2 <= end1) & (end1 <= end2): # overlap on left side of event 1
        ans = True
    elif (start1 <= start2) & (start2 <= end1): # event 2 contained in event 1
        ans = True
    else:
        ans = False

    return ans

for SOL_index, SOL_row in SOL_events.iterrows():
    SOL_id = SOL_row['SOL_standard'] 
    overlap = False  

    for csv_index, csv_row in csv_df.iterrows():
        csv_id = csv_row['SOL_event']

        if SOL_id == csv_id:
            """ compare time intervals """
            overlap = intervals_overlap(SOL_row['event_starttime'], SOL_row['event_endtime'], csv_row['date'], csv_row['end_time'])
        else:
            continue

    boolean_df.loc[len(boolean_df.index)] = overlap # True if the time intervals overlap

boolean_df



,overlap
0,False
1,True
2,True
3,True
4,False
...,...
4708,False
4709,False
4710,False
4711,True


In [36]:
final_df=SOL_events.copy()
final_df["overlap"]=boolean_df
final_df=final_df[final_df["overlap"]==False].reset_index()
final_df=final_df.drop(columns="index").rename(columns={"event_coord1": "basepoint_X", "event_coord2" : "basepoint_Y", "event_starttime": "date"})
final_df=final_df.sample(frac=1, random_state=42)
final_df=final_df[0:883]
final_df['date'] = pd.to_datetime(final_df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
display(final_df)
columns_to_export=["date", "basepoint_X", "basepoint_Y"]
final_df.to_csv("No_jet_df__.csv", columns=columns_to_export, index=False)



,SOL_standard,basepoint_X,basepoint_Y,date,event_endtime,overlap
718,SOL2011-07-07T00:00:04L335C080,432.0,118.0,2011-07-08T16:00:04.000000,2011-07-08 16:12:04,False
3116,SOL2016-02-18T00:00:06L069C082,888.0,190.0,2016-02-18T17:24:06.000000,2016-02-18 17:36:06,False
3061,SOL2016-02-18T00:00:06L069C082,888.0,190.0,2016-02-18T06:24:06.000000,2016-02-18 06:36:06,False
2552,SOL2015-08-06T21:30:34L046C100,-132.0,-279.0,2015-08-06T22:42:34.000000,2015-08-06 22:54:34,False
120,SOL2011-03-20T00:00:04L223C126,-790.0,-587.0,2011-03-20T01:48:04.000000,2011-03-20 02:00:04,False
...,...,...,...,...,...,...
222,SOL2011-04-01T03:00:11L222C115,836.0,-376.0,2011-04-01T03:12:11.000000,2011-04-01 03:24:11,False
969,SOL2012-04-02T00:30:04L075C088,1014.0,43.0,2012-04-02T04:18:04.000000,2012-04-02 04:30:04,False
1756,SOL2013-10-12T14:00:07L080C074,-951.0,278.0,2013-10-12T16:12:07.000000,2013-10-12 16:24:07,False
573,SOL2011-07-07T00:00:04L335C080,432.0,118.0,2011-07-07T10:36:04.000000,2011-07-07 10:48:04,False


In [ ]:
""" Importing 20 more events to have a balanced data set """
display(boolean_df)
add_boolean_df = boolean_df.copy()

for SOL_index, SOL_row in SOL_events.iterrows():
    SOL_id = SOL_row['SOL_standard'] 
    overlap = False  

    for final_index, final_row in final_df.iterrows():
        final_id = final_row['SOL_standard']

        if SOL_id == final_id:
            """ compare time intervals """
            overlap = intervals_overlap(SOL_row['event_starttime'], SOL_row['event_endtime'], final_row['date'], final_row['event_endtime'])
            
        else:
            continue

    if overlap == True:
        add_boolean_df.loc[len(add_boolean_df.index)] = overlap # True if the time intervals overlap




In [ ]:
additional_df = SOL_events.copy()
additional_df['overlap']=boolean_df
additional_df=final_df[final_df["overlap"]==False].reset_index()
additional_df=final_df.drop(columns="index").rename(columns={"event_coord1": "basepoint_X", "event_coord2" : "basepoint_Y", "event_starttime": "date"})
additional_df=final_df.sample(frac=1, random_state=42)
additional_df=final_df[0:883]
additional_df['date'] = pd.to_datetime(final_df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
display(additional_df)
columns_to_export=["date", "basepoint_X", "basepoint_Y"]
additional_df.to_csv("No_jet_add_df.csv", columns=columns_to_export, index=False)

<h1>Description of the features</h1>


<p><strong>"sjh_ID”</strong>: Solar Jet Hunter ID for this jet (data and hour of event; last number denotes which event this was in that hour) <br><br>
<strong>Date</strong>: Observation time of the start of the jet (format YYYY-MM-DDThh:mm:ss) <br><br>
<strong>SOL_event</strong>: The coronal jet event name from HEK database where this jet was found in (multiple jets can be found in one coronal event) <br><br>
<strong>Duration</strong> of the jet (minutes) <br><br>
<strong>Basepoint_X</strong>: Basepoint of the jet in (Heliocentric solar X coordinates in arcsec) <br><br>
<strong>Std_X</strong>: Standard deviation of the base point if there were multiple measurements of the base point (Heliocentric solar X coordinates in arcsec) <br> <br><strong>Basepoint_Y</strong>: Basepoint of the jet in (Heliocentric solar Y coordinates in arcsec)  <br><br>
<strong>Std_Y</strong>: Standard deviation of the base point if there were multiple measurements of the base point (Heliocentric solar Y coordinates in arcsec) <br> <br><strong>basepoint_X_longitude</strong>: Basepoint of the jet (longitude) <br><br>
<strong>basepoint_Y_latitude</strong>: Basepoint of the jet (latitude) <br><br>
<strong>max_height</strong>: Maximum height jet reaches (arcsec) <br><br>
<strong>upper_H</strong>: Upper limit of the error height (arcsec) <br><br>
<strong>lower_H</strong>: Lower limit of the error height (arcsec) <br><br
<strong>avg_width</strong>: Average width of the jet box (arcsec) <br><br>
<strong>std_width</strong>: Standard deviation width (arcsec) <br><br>
<strong>velocity</strong>: Velocity estimate (arcsec per sec)sigma: Average sigma of the jet, (sigma= 1-IoU) <br><br>
<strong>flag</strong>: Binary quality flagsflag 100 means the jet cluster has a duration of less than 6 minutes, which for many corresponds to a jet cluster found in one Zooniverse subject.flag 010 means the velocity estimate could not be calculated because the maximum was reached in the first subject the jet was found in.flag 001 means the basepoint has a Longitude of higher than 90 degrees meaning the base point was found to be (slightly) off limb. <br> </p>
